In [ ]:
from flask import Flask, jsonify, request, render_template
from flask_cors import CORS
import psycopg2

app = Flask(__name__)
CORS(app)

# Database connection parameters
db_params = {
    'host': '172.27.0.2',
    'port': 5432,
    'database': 'cartecredit',
    'user': 'postgres',
    'password': 'Postgres+123'
}


# Connect to the PostgreSQL database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Create the cards table if it doesn't exist
create_table_query = '''
    CREATE TABLE IF NOT EXISTS cards (
        id SERIAL PRIMARY KEY,
        card_number VARCHAR(16) NOT NULL,
        card_holder VARCHAR(255) NOT NULL,
        expiry_date VARCHAR(10) NOT NULL
    )
'''
cursor.execute(create_table_query)
conn.commit()

@app.route('/cards', methods=['GET'])
def get_all_cards():
    query = "SELECT * FROM cards"
    cursor.execute(query)
    cards = cursor.fetchall()
    return jsonify(cards)

@app.route('/cards', methods=['POST'])
def add_card():
    card = request.get_json()
    query = "INSERT INTO cards (card_number, card_holder, expiry_date) VALUES (%s, %s, %s)"
    values = (card['card_number'], card['card_holder'], card['expiry_date'])
    cursor.execute(query, values)
    conn.commit()
    return jsonify({'message': 'Card added successfully.'})

@app.route('/cards/<int:card_id>', methods=['GET'])
def get_card(card_id):
    query = "SELECT * FROM cards WHERE id = %s"
    values = (card_id,)
    cursor.execute(query, values)
    card = cursor.fetchone()
    if card:
        return jsonify(card)
    return jsonify({'message': 'Card not found.'}), 404

@app.route('/cards/<int:card_id>', methods=['PUT'])
def update_card(card_id):
    updated_card = request.get_json()
    query = "UPDATE cards SET card_number = %s, card_holder = %s, expiry_date = %s WHERE id = %s"
    values = (updated_card['card_number'], updated_card['card_holder'], updated_card['expiry_date'], card_id)
    cursor.execute(query, values)
    conn.commit()
    if cursor.rowcount > 0:
        return jsonify({'message': 'Card updated successfully.'})
    return jsonify({'message': 'Card not found.'}), 404

@app.route('/cards/<int:card_id>', methods=['DELETE'])
def delete_card(card_id):
    query = "DELETE FROM cards WHERE id = %s"
    values = (card_id,)
    cursor.execute(query, values)
    conn.commit()
    if cursor.rowcount > 0:
        return jsonify({'message': 'Card deleted successfully.'})
    return jsonify({'message': 'Card not found.'}), 404

@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Jun/2023 13:44:47] "POST /cards HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2023 13:44:52] "GET /cards HTTP/1.1" 200 -
[2023-06-23 13:44:59,377] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\aafathallah\Anaconda3\lib\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\aafathallah\Anaconda3\lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\aafathallah\Anaconda3\lib\site-packages\flask_cors\extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\aafathallah\Anaconda3\lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\aafathallah\Anaconda3\lib\site-packages\flask\app.py", line 1799, in dispatch_request
    